## Imports

In [ ]:
import torch
from torch import nn
from torch import Tensor
import torch.optim as optim
from torchvision import models, transforms, datasets

torch.cuda.is_available()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Visualize the data

In [ ]:
def plot_accuracy_from_history(history):
    plt.plot(history['accuracy'], label='accuracy')
    plt.plot(history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

# Transfer Learning

## Prepare Data

In [ ]:
EPOCHS: int = 10

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Define data directories
data_dir: str = 'kaggle/cats_and_dogs_small/'
image_datasets = {x: datasets.ImageFolder(root=data_dir+x, transform=data_transforms[x]) for x in ['train', 'validation', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=20, shuffle=False) for x in ['train', 'validation', 'test']}


## Features Extraction Method (faster but no augmentation)

### Define ResNet Model

In [ ]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer

    def forward(self, x):
        x = self.resnet(x)
        return x

### Extract features from ResNet

In [ ]:
def extract_features(model, dataloader, dataset_size) -> tuple[torch.Tensor, torch.Tensor]:
    model.eval()
    features = torch.zeros(dataset_size, 512, 4, 4, device=device)
    labels = torch.zeros(dataset_size, device=device)
    with torch.no_grad():
        for i, (inputs, labels_batch) in enumerate(dataloader):
            inputs = inputs.to(device)  # Move inputs to the same device as the model
            outputs = model(inputs)
            # Reshape or unsqueeze outputs to match the shape of features
            outputs = torch.unsqueeze(torch.unsqueeze(outputs, -1), -1)
            features[i * dataloader.batch_size : (i + 1) * dataloader.batch_size] = outputs
            labels[i * dataloader.batch_size : (i + 1) * dataloader.batch_size] = labels_batch
    return features, labels

In [ ]:
resnet_model = ResNetModel().to(device)

train_features, train_labels = extract_features(resnet_model, dataloaders['train'], len(image_datasets['train']))
validation_features, validation_labels = extract_features(resnet_model, dataloaders['validation'], len(image_datasets['validation']))
test_features, test_labels = extract_features(resnet_model, dataloaders['test'], len(image_datasets['test']))

# Reshape
train_features = train_features.view(len(image_datasets['train']), -1)
validation_features = validation_features.view(len(image_datasets['validation']), -1)
test_features = test_features.view(len(image_datasets['test']), -1)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

### New model

In [ ]:
class ModelWithExtracted(nn.Module):
    def __init__(self):
        super(ModelWithExtracted, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(train_features.shape[1], 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x

### Train

In [ ]:
model1 = ModelWithExtracted().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

def train_model(model, criterion, optimizer) -> None:
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum((outputs > 0.5) == labels.byte())
        epoch_loss = running_loss / len(image_datasets['train'])
        epoch_acc = running_corrects / len(image_datasets['train'])
        print(f"Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        
train_model(model1, criterion, optimizer)

## Fine-Tuning method (slower but with augmentation)

### Prepare Data

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Define data directories
data_dir = 'kaggle/cats_and_dogs_small/'
image_datasets = {x: datasets.ImageFolder(root=data_dir+x, transform=data_transforms[x]) for x in ['train', 'validation', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=20, shuffle=False) for x in ['train', 'validation', 'test']}


### Create model

In [ ]:
class ModelFineTuning(nn.Module):
    def __init__(self):
        super(ModelFineTuning, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        for param in self.resnet.parameters(): # freeze the ResNet layers
            param.requires_grad = False
        self.resnet.fc = nn.Identity()
        self.model = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.model(x)
        return x
    
    
model2 = ModelFineTuning().to(device)
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model2.parameters(), lr=2e-5)

In [ ]:
# Training loop
for epoch in range(EPOCHS):
    model2.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(image_datasets['train'])

    # Validation loop
    model2.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloaders['validation']:
            outputs = model2(inputs)
            val_loss = criterion(outputs, labels.float().view(-1, 1))
            val_running_loss += val_loss.item() * inputs.size(0)
    val_epoch_loss = val_running_loss / len(image_datasets['validation'])

    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}')